In [1]:
"""
    2018/03/15, K-Means PaperVersion with Outliers
    Local Search Methods for k-Means with Outliers, VLDB 2017
    from KMeans PaperVersion
"""

'\n    2018/03/15, K-Means PaperVersion with Outliers\n    Local Search Methods for k-Means with Outliers, VLDB 2017\n    from KMeans PaperVersion\n'

In [2]:
# -------- import --------

import math
import time
import copy
import random

# -------- parameters --------

k             = 3
z             = 2
dimension     = 4
threshold     = 1e-1
iterationTime = 10

path          = "Skin_NonSkin.txt"
splitChar     = "\t"

dataFromFile  = False # if True, read from files; else, create a random dataset (size)

# -------- variables -------- (version 0.1)

size          = 100
iterCount     = 0
centers       = []
costTotal     = 99999999

outliers      = []

timeBeg       = 0

In [3]:
# -------- create synthetic data -------- from KMeans PaperVersion (LS)

"""
k = 3, dimension = 4, clusterSize = [ , , ]
"""
def createData(n):
    listData = []
    for i in range(n):
        choice = random.randint(1,3)
        if choice == 1:
            newIn = [random.randint(0,10),random.randint(0,10),random.randint(10,20),random.randint(10,20),1]
            newIn = [float(newIn[a]) for a in range(4)] + [1]
            listData.append(newIn)
        elif choice == 2:
            newIn = [random.randint(50,60),random.randint(0,10),random.randint(10,30),random.randint(30,40),2]
            newIn = [float(newIn[a]) for a in range(4)] + [2]
            listData.append(newIn)
        else: 
            newIn = [random.randint(0,10),random.randint(20,30),random.randint(60,80),random.randint(100,200),3]
            newIn = [float(newIn[a]) for a in range(4)] + [3]
            listData.append(newIn)
    return listData

def createDataWO(n):
    listData = []
    for i in range(n):
        choice = random.randint(1,91)
        if choice <= 30:
            newIn = [random.randint(0,10),random.randint(0,10),random.randint(10,20),random.randint(10,20)]
            newIn = [float(newIn[a]) for a in range(4)] + [1]
            listData.append(newIn)
        elif choice <= 60:
            newIn = [random.randint(50,60),random.randint(0,10),random.randint(10,30),random.randint(30,40)]
            newIn = [float(newIn[a]) for a in range(4)] + [2]
            listData.append(newIn)
        elif choice <= 90:
            newIn = [random.randint(0,10),random.randint(20,30),random.randint(60,80),random.randint(100,200)]
            newIn = [float(newIn[a]) for a in range(4)] + [3]
            listData.append(newIn)
        else:
            newIn = [random.randint(80,90),random.randint(30,40),random.randint(60,80),random.randint(150,250)]
            newIn = [float(newIn[a]) for a in range(4)] + [-1]
            listData.append(newIn)
    return listData


In [4]:
# -------- functions --------

def clusterResult():
    """
    version 0.1 from KMeans PV: show the origin cluster: size of the ith cluster
    version 0.3 modified: fit new data type
    rename as clusterResult (used to be originResult)
    """
    outlierCount = size
    for i in range(k):
        csize = data.filter(lambda x: x[1]==i+1).count()
        outlierCount = outlierCount - csize
        print("Origin cluster " + str(i+1) + " : " + str(csize))
    print("Outliers: " + str(outlierCount))
    if outlierCount>0:
        o = data.filter(lambda x:x[1]==-1).collect()
        print o

def costKM():
    """
    version 0.1 from KMeans PV: show the total cost of the cluster with center set centerTemp
    version 0.2 discard some outliers (lambda x: x[-1]== -1)
    version 0.3 discard some outliers(RDD), now x[-1] is the cost
        costSum = data.filter(lambda x: x[1]!= -1).map(lambda x: x[-1]).reduce(lambda x,y: x+y)
    version 0.4 now we assume that all outliers have a 0 cost
    """
    costSum = data.map(lambda x: x[-1]).reduce(lambda x,y: x+y)
    return costSum

def updateCluster(u,centerTemp):
    """
    version 0.3 find a cluster for just one point and return [clusterNum,costMin]
    version 0.4 
        data = data.map(lambda x: [[x[0]]]+updateCluster(x,centerTemp) if x[1]!= -1 else x)
        is replaced by data = data.map(lambda x: [[x[0]]]+updateCluster(x,centerTemp))
        hence, if x[1]==-1, we return [-1,0] in this function
    """
    if u[1] == -1:
        return [u[0],-1,0]
    costMin = 9999999
    pos     = -1
    for i in range(k):
        A = u[0]
        B = centerTemp[i][0]
        costTemp = sum([(A[j]-B[j])**2 for j in range(dimension)])
        if costTemp < costMin:
            costMin = costTemp
            pos = i
    return [u[0],pos+1,costMin]

def costKMTemp(centerTemp):
    # centerTemp are supposed to be new centers
    # sum(x[i]-c[i]**2 for i in range(dimension)) is the distance between two points
    # min(distance(x,c) for c in centerTemp) is the distance between a point and center points
    costSum = data.filter(lambda x: x[1]>0 ).map(lambda x:(min(sum((x[0][i]-c[0][i])**2 for i in range(dimension)) for c in centerTemp))).reduce(lambda x,y : x+y)
    return costSum
    
def compareCenters(cA,cB,p):
    """version 0.1 from KMeans PaperVersion, compare centers before and after
            now we have two set of centers, cA=[c1A,c2A,c3A...], ciA=[d1,d2,d3...]
            int(di*10^p), for example, if p==2, 16.87!=16.88 && 16.887==16.888
       version 0.3 fit new data
    """
    for i in range(k):
        ciA = cA[i][0]
        ciB = cB[i][0]
        for j in range(dimension):
            if int(ciA[j]*10**p)-int(ciB[j]*10**p) >=1:
                return False
    return True

def swap(A,centerTemp,i):
    """version 0.1 from KMeans PaperVersion, perform a swap.
        the swap is performed by replacing C by (C+[u])\[v] (in fact for each u in U and each v in C)"""
    return centerTemp[:i] + [A] + centerTemp[i+1:]

def isOutlier(x,standardValue):
    """version 0.5 if the cost of a point is bigger than standardValue, it's a outlier point
            for those outliers, x[-1]>=standardValue, set x[1]=-1, x[-1]=0
    """
    if (x[-1] < standardValue):
        return x
    else:
        return [x[0],-1,0]
    
def isOutlierTemp(x,standardValue):
    """version 0.5 we have a standard value, but we haven't decide to discard these points permanently.
            if the cost of a point is bigger than standardValue, it's a outlier point
            for those outliers, x[-1]>=standardValue, set x[1]=0, x[-1]=0
    """
    if (x[-1] < standardValue):
        return x
    else:
        return [x[0],0,0]
    
def confirmOutlier(x):
    """check isOutlierTemp
       version 0.6 x[1]==0 -> x[1]=-1
    """
    if (x[1]!=0):
        return x
    else:
        return [x[0],-1,0]

def updateCost(u,centerTemp):
    """
    version 0.5 from updateCluster, only return new cost, discard x[1] < 1 
    """
    if u[1] < 1:
        return 0
    costMin = 9999999
    for i in range(k):
        A = u[0]
        B = centerTemp[i][0]
        costTemp = sum([(A[j]-B[j])**2 for j in range(dimension)])
        if costTemp < costMin:
            costMin = costTemp
    return costMin    
    
def costWO(centerTemp):
    """
    version 0.5 for part 3, a new center set with outliers
    version 0.6 updateCost return 0 for outliers(lambda x:x[1]<1), hence we don't need .filter() here
    """
    newData = data.map(lambda x: updateCost(x,centerTemp))
    costSum = newData.reduce(lambda x,y:x+y) - sum(newData.top(z))
    return costSum

In [5]:
# -------- create RDD -------- from KMeans PaperVersion (LS)

""" version 0.3
        line            = [d0,d1...clusterNo]
        x               = [line[:-1],clusterNo,cost]
        clusterNo       = line[-1] = x[1]   
        cost(x,centers) = x[2] = 999999(temporary) """

if dataFromFile :
    # 1. from files (real data)
    data = sc.textFile(path).map(lambda line: line.split(splitChar)).map(lambda x : [float(a) for a in x[:-1]],int(x[-1]), 999999)
else:
    # 2. random lists (synthetic data) (version 0.2)
    datalist = createDataWO(size)
    datalist = [[x[:-1],x[-1],0] for x in datalist]
    print datalist[1]
    data = sc.parallelize(datalist)

clusterResult()

size = data.count()
print("All " + str(size) + " points." )

# start timing
timeBeg = time.time()


[[5.0, 5.0, 10.0, 19.0], 1, 0]
Origin cluster 1 : 162
Origin cluster 2 : 160
Origin cluster 3 : 170
Outliers: 8
[[[90.0, 38.0, 77.0, 155.0], -1, 0], [[86.0, 40.0, 63.0, 166.0], -1, 0], [[86.0, 39.0, 61.0, 154.0], -1, 0], [[84.0, 32.0, 71.0, 212.0], -1, 0], [[87.0, 31.0, 65.0, 170.0], -1, 0], [[90.0, 33.0, 72.0, 197.0], -1, 0], [[90.0, 40.0, 70.0, 151.0], -1, 0], [[86.0, 36.0, 70.0, 150.0], -1, 0]]
All 500 points.


In [6]:
# -------- pick centers --------

# an arbitrary set of k points from U
centers = data.takeSample(False,k,random.randint(1,1000))
print(centers)

# K-means++ (k+2z)
# TODO ####################################################

data = data.map(lambda x: updateCluster(x,centers))

# -------- compute outliers --------
# outliers : lambda x : x[-1] == -1, x[-1] == 0 
# (cost=0 means that we dont need to .filter() before sum up)
# sort all cost and set standard
standard = (data.map(lambda x:x[-1]).top(z))[-1]
data = data.map(lambda x: isOutlier(x,standard))

""" # wanna know whether we've found z outliers or not, try these codes:
outlierCount = data.filter(lambda x:x[1]==-1).count()
print outlierCount """

[[[52.0, 1.0, 12.0, 34.0], 2, 0], [[2.0, 26.0, 77.0, 121.0], 3, 0], [[7.0, 20.0, 62.0, 130.0], 3, 0]]


" # wanna know whether we've found z outliers or not, try these codes:\noutlierCount = data.filter(lambda x:x[1]==-1).count()\nprint outlierCount "

In [7]:
# -------- iterations --------
"""
we already have:
data(RDD), centers[], outliers[],costTotal(Float) 
as final results.

new variables:

as temporary results.
"""
costTotal = costKM()

while (iterCount < iterationTime):
    
    costTemp = costTotal
    
    ########################################################
    # -------- part 1 local search with no outliers --------
    # costTotal1,centers1,centerN,costSwap,costTemp
    centers1 = copy.deepcopy(centers)
    
    while True :
        # update costTotal1
        costTotal1 = costKM() #result of last iteration
        print("costTotal1"+ str(costTotal1))
        # update cluster data in RDD
        data = data.map(lambda x: updateCluster(x,centers1))

        centerN = copy.deepcopy(centers1)
        # a temporary improved set of centers1
        for i in range(k):
            tempCluster = data.filter(lambda x: x[1]==i+1 ).map(lambda x:x[0])
            sizeCluster = tempCluster.count()
            sumCluster  = tempCluster.reduce(lambda x,y:[x[i]+y[i] for i in range(dimension)])
            centerN[i]  = [[sumCluster[h]/sizeCluster for h in range(dimension)],i+1,0]
        costTemp = costKMTemp(centerN)
        
        # break immediately
        if(compareCenters(centerN,centers1,2)):
            break
            
        # for each center and non-center, perform a swap
        listAllPoints = data.filter(lambda x: x[1]!= -1).collect()
        costSwap      = costTemp
        for u in listAllPoints:
            swapF = -1
            for swapPos in range(k):
                centerSwap = swap(u,centerN,swapPos)
                costSwapT = costKMTemp(centerSwap)
                if costSwapT < costSwap :
                    swapF = swapPos
                    costSwap = costSwapT
            if swapF != -1:
                centerN = swap(u,centerN,swapF)
        del listAllPoints

        costTemp = costKMTemp(centerN)
        if costTemp < (1-threshold)*costTotal1:
            costTotal1 = costTemp
            centers1 = copy.deepcopy(centerN)
            data = data.map(lambda x: updateCluster(x,centers1))
        else:
            break      
    # now, centers1 is a temporary improved set of centers, costTotal1 is the improved cost
    print centers1
        
    ###################################################################
    # -------- part 2 cost of discarding z additional outliers --------
    """there is a problem: a outlier point x should belong to cluster -1, we will calculate cost without outliers
       but we havent decide whether 'outliers' in this part should be outliers or not
       therefore, we set these x: x[1] = 0 instead."""
    # find new temporary outliers (cost)
    outlierTemp = data.map(lambda x:x[-1]).top(z)
    print ("OUTLIER TEMP")
    print outlierTemp
    costTotal2 = costTotal1
    if sum(outlierTemp) > threshold*costTotal1:
        # costTotal2 = costTotal1 - sum(...)
        # ostTotal2 < (1-threshold)* costTotal1 <=> sum(...) > threshold*costTotal
        data = data.map(lambda x: isOutlierTemp(x,outlierTemp[-1])) # add some temporary outliers
        costTotal2 = costTotal1 - sum(outlierTemp)  # <=> costKM()
    # centers1 is still the set of centers, costTotal2 is the improved cost
    
    ########################################################################################################
    # -------- part 3 for each center an non-center, perform a swap and discard additional outliers --------
    
    listAllPoints = data.filter(lambda x: x[1]>=1).collect()
    centerN = copy.deepcopy(centers1)
    costTotal3 = costTotal2
    
    for u in listAllPoints:
        for swapPos in range(k):
            centerSwap = swap(u,centers1,swapPos)
            costSwapT = costWO(centerSwap)
            if costSwapT < costTotal3 :
                costTotal3 = costSwapT
                centerN = copy.deepcopy(centerSwap)
    # centerN is centers with the most improved swap
    if(costTotal3!=costTotal2): # if there's a swap
        # update data (updateCluster update those x[1]!=-1, but there exists some temporary outliers)
        data = data.map(lambda x: x if x[1]==0 else updateCluster(x,centerN))
        # calculate new outliers, x[1]=0 (temporary outliers)
        standardTemp = data.map(lambda x:x[-1]).top(z)
        data = data.map(lambda x: isOutlierTemp(x,standardTemp[-1]))
        # calculate new cost
        costTotal3 = costKM()
    print("TIME")
    
    ######################################
    # -------- part 4 final check --------
    # update the solution allowing additional outliers if the solution value improved significantly
    if ( costTotal3 < (1-threshold)*costTotal ):
        centers = copy.deepcopy(centerN)
        data = data.map(lambda x: confirmOutlier(x))
        data = data.map(lambda x: updateCluster(x,centers))
        costTotal = costTotal3
    else:
        data = data.map(lambda x: updateCluster(x,centers))
        break
    
    iterCount += 1

costTotal1697536.0
costTotal1162489.0
costTotal1159223.98534
[[[55.05625, 5.04375, 19.6, 35.2375], 1, 0], [[4.660493827160494, 5.166666666666667, 14.697530864197532, 15.333333333333334], 2, 0], [[4.8273809523809526, 25.357142857142858, 70.16666666666667, 153.80357142857142], 3, 0]]
OUTLIER TEMP
[2740.890376984126, 2722.4499007936497]
costTotal1153467.686508
costTotal1147538.258468
[[[55.05625, 5.04375, 19.6, 35.2375], 1, 0], [[4.660493827160494, 5.166666666666667, 14.697530864197532, 15.333333333333334], 2, 0], [[4.771084337349397, 25.39156626506024, 70.43373493975903, 155.5722891566265], 3, 0]]
OUTLIER TEMP
[2280.7213673973006, 2204.4683553491077]
costTotal1139161.186103
costTotal1134087.949845
[[[55.05625, 5.04375, 19.6, 35.2375], 1, 0], [[4.660493827160494, 5.166666666666667, 14.697530864197532, 15.333333333333334], 2, 0], [[4.81875, 25.325, 70.4375, 157.3375], 3, 0]]
OUTLIER TEMP
[2004.3187890625004, 1937.6187890625004]
costTotal1133727.131016
costTotal1116074.278953


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kana/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/kana/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/kana/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/kana/anaconda2/lib/python2.7/inspect.py", line 1051, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/home/kana/anaconda2/lib/python2.7/inspect.py", line 1011, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/kana/anaconda2/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/home/kana

IndexError: string index out of range

In [ ]:
timeEnd = time.time()
print("Time: " + str(timeEnd-timeBeg))

In [ ]:
clusterResult()

In [ ]:
print costKM()